# Gensim LDA model for Japanese articles
auther : m3yrin

reference : http://tdual.hatenablog.com/entry/2018/04/09/133000

### Memo
* tdual' s LDA script is massively cited.
* janome tokenizer is used instead of Mecab.

### Dataset
livedoor ニュースコーパス / livedoor News Corpus  
https://www.rondhuit.com/download.html#ldcc  
CC BY-ND 2.1 JP  
https://creativecommons.org/licenses/by-nd/2.1/jp/

In [1]:
!pip install janome

import os
if not os.path.exists('text'):
    !wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
    !tar xvzf ldcc-20140209.tar.gz

  Using cached Janome-0.4.2-py2.py3-none-any.whl (19.7 MB)


In [2]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import re
import janome
import random
from gensim import corpora, models

from janome.tokenizer import Tokenizer
from janome import analyzer
from janome.charfilter import *
from janome.tokenfilter import *

from tqdm import tqdm_notebook as tqdm

In [8]:
# https://ohke.hateblo.jp/entry/2017/11/02/230000
class NumericReplaceFilter(TokenFilter):
    def apply(self, tokens):
        for token in tokens:
            parts = token.part_of_speech.split(',')
            if (parts[0] == '名詞' and parts[1] == '数'):
                token.surface = '0'
                token.base_form = '0'
                token.reading = 'ゼロ'
                token.phonetic = 'ゼロ'
            yield token

            
class docTokenizer:
    def __init__(self, stopwords, parser=None, include_pos=None, exclude_posdetail=None, exclude_reg=None):
    
        self.stopwords = stopwords
        self.include_pos = include_pos if include_pos else  ["名詞", "動詞", "形容詞"]
        self.exclude_posdetail = exclude_posdetail if exclude_posdetail else ["接尾", "数"]
        self.exclude_reg = exclude_reg if exclude_reg else r"$^"  # no matching reg
        
        self.char_filters = [
                        UnicodeNormalizeCharFilter(), 
                        RegexReplaceCharFilter(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+", u''), #url
                        RegexReplaceCharFilter(r"\"?([-a-zA-Z0-9.`?{}]+\.jp)\"?", u''), #*.jp
                        RegexReplaceCharFilter(self.exclude_reg, u'')
                       ]
        
        self.token_filters = [
                         NumericReplaceFilter(),
                         POSKeepFilter(self.include_pos),
                         POSStopFilter(self.exclude_posdetail), 
                         LowerCaseFilter()
                        ]
        
        self.analyzer = analyzer.Analyzer(char_filters=self.char_filters, tokenizer=Tokenizer(), token_filters=self.token_filters)
        
    def tokenize(self, text):

        tokens = self.analyzer.analyze(text)
        tokens = [re.sub(r"," ,"\t", str(i)) for i in tokens]
        l = [line.split("\t") for line in tokens]
        
        #Janome response
        #i[] : ['認め', '動詞', '自立', '*', '*', '一段', '連用形', '認める', 'ミトメ', 'ミトメ']

        res = []
        for i in l:
            if i[7] not in self.stopwords:
                res.append(i[7])
                
        return res
        


### Hyper-parameters

In [9]:
num_articles = -1
topic_num = 20
passes = 50

### Load stopwords

In [10]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/English.txt")
stopwords += [line.decode("utf-8").strip() for line in res]

stopwords += ['*', '&', '[', ']', ')', '(', '-',':','.','/','0', '...?', '——', '!【', '"', ')、', ')。', ')」']

print("# Stopword : ", len(stopwords))

# Stopword :  928


### Load articles

In [11]:
doc_path = "./text/"
doc_dir = Path(doc_path)
dirs = [i for i in doc_dir.iterdir() if i.is_dir()]
articles = [a for categ in dirs for a in categ.iterdir()]
random.shuffle(articles)

articles = articles[:num_articles]

In [14]:
df_exp = pd.read_csv('/home/yamanishi/project/trip_recommend/data/jalan/spot/experience_light.csv')
df_review = pd.read_csv('/home/yamanishi/project/trip_recommend/data/jalan/review/review_all.csv')
train_spot = df_exp[df_exp['valid']>=2].loc[:,'spot_name']
valid_spot = df_exp[df_exp['valid']==1].loc[:, 'spot_name']
test_spot = df_exp[df_exp['valid']==0].loc[:, 'spot_name']
df_review_train = df_review[df_review['spot'].isin(train_spot)]
len(df_review_train)

/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/study_group/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


1069153

In [18]:
tokenizer = docTokenizer(stopwords = stopwords, exclude_reg=r"\d(年|月|日)")

docs = []
for a in tqdm(df_review_train['review']):
    docs.append(tokenizer.tokenize(a))
    """
    with a.open() as f:
        
        # discard first two lines.
        f.readline()
        f.readline()
        docs.append(tokenizer.tokenize(f.read()))
    """

/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/study_group/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/1069153 [00:00<?, ?it/s]

### Build dict

In [ ]:
# build dict
d = corpora.Dictionary(docs)
d.filter_extremes(no_below=5, no_above=0.2)
d.compactify()

# make bow
corpus = [d.doc2bow(w) for w in docs]

# test train split
test_size = int(len(corpus) * 0.1)
test_corpus = corpus[:test_size]
train_corpus = corpus[test_size:]

### Build LDA

In [ ]:
# logging setting
logging.basicConfig(format='%(message)s', level=logging.INFO)

# build LDA
lda = models.LdaModel(corpus=train_corpus, id2word=d,num_topics=topic_num,passes=10, update_every=5)

### Results

In [ ]:
N = sum(count for doc in train_corpus for id, count in doc)
print("# of words in train corpas: ",N)
perplexity = np.exp2(-lda.log_perplexity(train_corpus))
print("perplexity(train):", perplexity)

print("==============================")
N = sum(count for doc in test_corpus for id, count in doc)
print("# of words in test corpas: ",N)
perplexity = np.exp2(-lda.log_perplexity(test_corpus))
print("perplexity(test):", perplexity)

In [ ]:
def get_topic_words(topic_id):
    tw = []
    for t in lda.get_topic_terms(topic_id):
        tw.append(d[t[0]])
    
    return tw

for t in range(topic_num):
    tw = get_topic_words(t)
    print('Topic {}: {}'.format(t + 1, ' '.join(tw)))